## Código 

In [ ]:
#include <NewPing.h> // Biblioteca para el sensor ultrasónico
#include <Wire.h> // Biblioteca para la comunicación I2C
#include <LiquidCrystal_I2C.h> // Biblioteca para la pantalla LCD I2C
#include <Keypad.h> // Biblioteca para el teclado numérico
#include <stdlib.h> // Biblioteca estándar de C++

LiquidCrystal_I2C lcd(0x27, 16, 2); // Crea el objeto LCD con la dirección I2C y el tamaño de la pantalla
NewPing nivel(8, 9, 100); // Crea el objeto para el sensor ultrasónico, con los pines de trigger y echo, y la distancia máxima
int porcentajeHumedadEstablecido; // Variable para almacenar el porcentaje de humedad establecido
int porcentajeHumedadMedido; // Variable para almacenar el porcentaje de humedad medido
int nivelAguaMedido; // Variable para almacenar el nivel de agua medido
const int electroValvulaPin = 12; // Pin de la electroválvula
const int bombaPin = 13; // Pin de la bomba

char keys[4][3] = { // Definición de los caracteres del teclado numérico
  {'1', '2', '3'},
  {'4', '5', '6'},
  {'7', '8', '9'},
  {'*', '0', '#'}
};

byte rowPins[4] = {7, 6, 5, 4}; // Pines utilizados para cada fila del teclado
byte colPins[3] = {3, 2, 1}; // Pines utilizados para cada columna del teclado

Keypad keypad = Keypad(makeKeymap(keys), rowPins, colPins, 4, 3); // Crea el objeto para el teclado numérico

int teclado() // Función para leer el valor ingresado por el teclado
{
    int posicion = 0;
    String cadena = "";
    char key = NO_KEY;

    lcd.setCursor(0, 0);
    lcd.print("Ingresar valor:"); // Imprime un mensaje en la primera fila del LCD

    while (key != '#')
    {
        key = keypad.getKey(); // Lee la tecla presionada

        if (key != NO_KEY)
        {
            if(posicion == 1 && key == '#')
            {
                posicion = 2;
            }

            if (key == '*')
            {
                lcd.clear(); // Borra la pantalla si se presiona la tecla '*'
                posicion = 0;
                cadena = "";
                lcd.setCursor(0, 0);
                lcd.print("Ingresar valor:");
            }

            else if (posicion < 2 && key != '#')
            {
                lcd.setCursor(posicion, 1); // Mueve el cursor a la columna especificada en 'posicion'
                lcd.print(key); // Imprime la tecla presionada en el LCD
                cadena.concat(key); // Concatena el carácter a la cadena de valores
                posicion = posicion + 1;
            }
        }
    }

    if(cadena!="")
    {
        int porcentaje = atoi(cadena.c_str()); // Convierte la cadena a un entero
        Serial.println(porcentaje); // Imprime el valor en el Monitor Serial
        lcd.setCursor(0, 0);
                lcd.print("                "); // Imprime 16 espacios en blanco para borrar la línea
        lcd.setCursor(0, 0);
        lcd.print("Confirmado");
        return porcentaje; // Devuelve el valor ingresado por el teclado
    }
}

void setup()
{
    lcd.init(); // Inicializa el LCD
    lcd.backlight(); // Enciende el backlight del LCD
    pinMode(bombaPin, OUTPUT); // Configura el pin de la bomba como salida
    pinMode(electroValvulaPin, OUTPUT); // Configura el pin de la electroválvula como salida
    digitalWrite(bombaPin, LOW); // Apaga la bomba
    digitalWrite(electroValvulaPin, LOW); // Cierra la electroválvula
    porcentajeHumedadEstablecido = teclado(); // Obtiene el valor del porcentaje de humedad establecido desde el teclado
    Serial.begin(9600); // Inicia la comunicación serial
}

void loop()
{
    nivelAguaMedido = nivel.ping_cm(); // Mide la distancia del sensor ultrasónico al agua

    if(nivelAguaMedido > 4)
    {
        digitalWrite(electroValvulaPin,HIGH); // Abre la electroválvula si el nivel de agua es mayor a 4 cm
        lcd.setCursor(0, 0);
        lcd.print("                "); // Imprime 16 espacios en blanco para borrar la línea
        lcd.setCursor(0, 0);
        lcd.print("Paso activado"); // Imprime en el LCD que el paso está activado
    }
    else
    {
        digitalWrite(electroValvulaPin,LOW); // Cierra la electroválvula si el nivel de agua es menor o igual a 4 cm
        lcd.setCursor(0, 0);
        lcd.print("                "); // Imprime 16 espacios en blanco para borrar la línea
        lcd.setCursor(0, 0);
        lcd.print("Paso desactivado"); // Imprime en el LCD que el paso está desactivado
    }

    int humidity = analogRead(A0); // Lee el valor de humedad del sensor

    if (humidity >= 350 && humidity <= 985)
    {
        porcentajeHumedadMedido = map(humidity, 985, 350, 0, 100); // Mapea el valor de humedad a un rango de 0 a 100
    }
    else
    {
        if (humidity > 985)
        {
            porcentajeHumedadMedido = 0; // Establece el valor de humedad medido en 0 si está por encima del rango esperado
        }
        else if (humidity < 350)
        {
            porcentajeHumedadMedido = 100; // Establece el valor de humedad medido en 100 si está por debajo del rango esperado
        }
    }

    if(porcentajeHumedadMedido < porcentajeHumedadEstablecido)
    {
        digitalWrite(bombaPin,HIGH); // Enciende la bomba si el porcentaje de humedad medido es menor al establecido
        lcd.setCursor(0, 0);
        lcd.print("                "); // Imprime 16 espacios en blanco para borrar la línea
        lcd.setCursor(0, 0);
        lcd.print("Riego activado"); // Imprime en el LCD que el riego está activado
    }
    else
    {
        digitalWrite(bombaPin,LOW); // Apaga la bomba si el porcentaje de humedad medido es mayor o igual al establecido
        lcd.setCursor(0, 0);
        lcd.print("                "); // Imprime 16 espacios en blanco para borrar la línea
        lcd.setCursor(0, 0);
        lcd.print("Riego desactivado"); // Imprime en el LCD que el riego está desactivado
    }

    delay(500); // Espera 500 milisegundos antes de repetir el ciclo
}